In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()
from Tools import mean_st,replace_punctuation, plot_predict

%matplotlib inline

Loading BokehJS ...

In [2]:
ruta="/home/comdisde/Documentos/Data/AccionesMexico/"

In [3]:
for archivo in os.listdir(ruta):
    if 'Datos hist\xc3\xb3ricos ' in archivo:
        os.rename(ruta+archivo, ruta+archivo.replace('Datos hist\xc3\xb3ricos ',""))
        print archivo.replace('Datos hist\xc3\xb3ricos ',"")
    
    
    nmb=archivo.replace(".csv","")
    
    exec("df_%s=pd.read_csv('%s')[['Fecha','Cierre','Apertura','Vol.']]" %(nmb,ruta+archivo))
    
    
    
    exec("df_%s['Fecha']=pd.to_datetime(df_%s['Fecha'].map(lambda x: (x[3:5]+'/'+x[:2]+'/'+x[-4:])))" %(nmb
                                                           ,nmb))
    
    exec("df_%s['Dia']=df_%s.Fecha.dt.day" %(nmb,nmb))
    exec("df_%s['Mes']=df_%s.Fecha.dt.month" %(nmb,nmb))
   

    exec('df_%s["Vol."]=df_%s["Vol."].iloc[:-1].map(lambda x: float(x[:-1])*1000 if x[-1]=="K" else float(x[:-1])*1000000)'
        %(nmb,nmb))
    
    exec("aux=df_%s" %nmb)
    aux["Vol. anterior"]=[np.nan]+[x for x in aux["Vol."].iloc[:-1]]
    aux["Cierre anterior"]=[np.nan]+[x for x in aux["Cierre"].iloc[:-1]]
    
    control="Error_Cierrre"
    
    try:
        aux.Cierre=aux.Cierre.map(replace_punctuation).map(lambda x: x[:-2]+"."+x[-2:]).astype(float)
    except: control= ""
    
    exec('df_%s=mean_st(dfr=df_%s, target="Cierre",n=[2,5,7,10]).iloc[10:].reset_index(drop=True)' 
         %(nmb,nmb))
    
    exec("""df_%s=df_%s[['Fecha','Vol.',"Apertura", "Dia", "Mes", "Vol. anterior", 
               "Cierre anterior"]+[x for x in df_C.columns if "ob" in x]+["Cierre"]]""" %(nmb,nmb))
    
    
    for k in range(1,4):
        exec('df_%s["Cierre_dia%d"]=df_%s.Cierre.tolist()[%d:]+[np.nan for x in range(%d)]' 
             %(nmb, k+1,nmb , k, k) )
    
    
    exec('df_%s.rename(columns={"Cierre":"Cierre_dia1"}, inplace=True)' %nmb)
    #del aux
    print "df_%s" %(nmb), control,
    exec("print df_%s.shape" %(nmb))
    

df_C Error_Cierrre (660, 19)
df_BSMXB  (663, 19)
df_CMXCPO  (663, 19)
df_ALSEA  (663, 19)
df_PENOLES  (663, 19)
df_BIMBOA  (663, 19)


In [40]:
from sklearn.preprocessing import  MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error
from xgboost import XGBRegressor

In [41]:
Cierres=["Cierre_dia%d" %x for x in range(1,5)]

In [62]:
dic_modelos={}
for dia in range(len(Cierres)): 
    df=df_BIMBOA.copy()
    
    if dia==0:X=df[[x for x in df.columns if not("Cierre_dia") in x]].iloc[:,2:].values
    else: X=df[[x for x in df.columns if not("Cierre_dia") in x]].iloc[:-dia,2:].values
    
    y=df[["Cierre_dia%d" %(dia+1)]].dropna().values.reshape(-1,1)
    
    print X.shape,y.shape

    
    scx = MinMaxScaler()
    scy = MinMaxScaler()
    scx.fit(X)
    scy.fit(y)
    
    Xs = scx.transform(X)
    ys = scy.transform(y)
    
    Xt,Xv,yt,yv = train_test_split(Xs,ys,train_size=0.7, random_state=0)
    
    dic_modelos["X_dia%d" %(dia+1)]=X
    dic_modelos["y_dia%d" %(dia+1)]=y 
    dic_modelos["Xs_dia%d" %(dia+1)]=Xs
    dic_modelos["ys_dia%d" %(dia+1)]=ys
    dic_modelos["scx_dia%d" %(dia+1)]=scx
    dic_modelos["scy_dia%d" %(dia+1)]=scy
    model=XGBRegressor()
    model.fit(Xt,yt)
    print"Error con muestra de entrenamiento: {0:.2f}%".format(100*r2_score(y_pred=model.predict(Xt),y_true=yt)) 
    print "Error con muestra de prueba: {0:.2f}%".format(100*r2_score(y_pred=model.predict(Xv),y_true=yv))

    

(663, 13) (663, 1)
Error con muestra de entrenamiento: 99.38%
Error con muestra de prueba: 97.52%
(662, 13) (662, 1)
Error con muestra de entrenamiento: 99.04%
Error con muestra de prueba: 96.31%
(661, 13) (661, 1)
Error con muestra de entrenamiento: 98.63%
Error con muestra de prueba: 95.80%
(660, 13) (660, 1)
Error con muestra de entrenamiento: 98.56%
Error con muestra de prueba: 94.98%


In [134]:
model=XGBRegressor()
model.fit(Xt,yt)
print"Error con muestra de entrenamiento: {0:.2f}%".format(100*r2_score(y_pred=model.predict(Xt),y_true=yt)) 
print "Error con muestra de prueba: {0:.2f}%".format(100*r2_score(y_pred=model.predict(Xv),y_true=yv))


Error con muestra de entrenamiento: 99.38%
Error con muestra de prueba: 97.52%


In [136]:
plot_predict(y_real=scy.inverse_transform(yt[dia]).reshape(1,-1)[0], 
             y_pred=scy.inverse_transform(pd.DataFrame(model.predict(Xt))).reshape(1,-1)[0], 
             title="BIMBO Train")

In [137]:
plot_predict(y_real=scy.inverse_transform(yv[dia]).reshape(1,-1)[0], 
             y_pred=scy.inverse_transform(pd.DataFrame(model.predict(Xv))).reshape(1,-1)[0], 
             title="BIMBO Validate")

In [138]:
plot_predict(y_real=scy.inverse_transform(ys[dia]).reshape(1,-1)[0], 
             y_pred=scy.inverse_transform(pd.DataFrame(model.predict(Xs))).reshape(1,-1)[0], 
             title="BIMBO Validate", circle=False)

In [193]:

Obv_mean=lambda n: df.iloc[-n:][dia].mean()
Obv_std=lambda n: df.iloc[-n:][dia].std()
Cierre_anterior=df.tail(1)[dia].iloc[0]
Volumen_anterior=df.tail(1)["Vol."].iloc[0]
Dia_manana=df.tail(1)["Dia"].iloc[0]+1




predict=np.array([39.995, 4, 9,3430000.0,39.995 
        ,Obv_std(2), Obv_mean(2),Obv_std(5), Obv_mean(5), Obv_std(7), 
         Obv_mean(7), Obv_std(10), Obv_mean(10) ]).reshape(-1,1)

prediccion=scy.inverse_transform(model.predict(pd.DataFrame(scx.transform(predict.T), 
                columns=X.columns)).reshape(-1,1))[0][0]

In [194]:
Obv_mean(2)

Cierre_dia1    39.8725
dtype: float64

In [195]:
print "El precio de cierre para mañana es: %f" %prediccion

El precio de cierre para mañana es: 39.784775
